In [1]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("Reading from file").master("local[*]").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/04/20 13:48:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/20 13:48:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/20 13:48:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/04/20 13:48:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/04/20 13:48:31 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/04/20 13:48:31 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/04/20 13:48:31 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


In [6]:
df_single_json_df=spark.read.format("json").load("input/order_singleline.json")
df_single_json_df.show(truncate=False)
df_single_json_df.printSchema()

+------------------------+-----------+--------+------------------------------------+
|contact                 |customer_id|order_id|order_line_items                    |
+------------------------+-----------+--------+------------------------------------+
|[9000010000, 9000010001]|C001       |O101    |[{102.45, I001, 6}, {2.01, I003, 2}]|
+------------------------+-----------+--------+------------------------------------+

root
 |-- contact: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_line_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: double (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- qty: long (nullable = true)



In [7]:
%%markdown

## reading multiline json


## reading multiline json


In [9]:
df_multiline_json_df=spark.read.format("json").option("multiline",True).load("input/order_multiline.json")
df_multiline_json_df.show(truncate=False)
df_multiline_json_df.printSchema()

+------------------------+-----------+--------+------------------------------------+
|contact                 |customer_id|order_id|order_line_items                    |
+------------------------+-----------+--------+------------------------------------+
|[9000010000, 9000010001]|C001       |O101    |[{102.45, I001, 6}, {2.01, I003, 2}]|
+------------------------+-----------+--------+------------------------------------+

root
 |-- contact: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_line_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: double (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- qty: long (nullable = true)



In [12]:
df=spark.read.format("text").option("multiline",True).load("input/order_singleline.json")
df.show(truncate=False)
df.printSchema()

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"order_id":"O101","customer_id":"C001","order_line_items":[{"item_id":"I001","qty":6,"amount":102.45},{"item_id":"I003","qty":2,"amount":2.01}],"contact":[9000010000,9000010001]}|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

root
 |-- value: string (nullable = true)



In [13]:
%%markdown

#### writing the schema for complex structure


#### writing the schema for complex structure


In [17]:
_schema="customer_id string, order_id String, contact array<long>"

df_with_schema=spark.read.format("json").schema(_schema).load("input/order_singleline.json")
df_with_schema.printSchema()
df_with_schema.show()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- contact: array (nullable = true)
 |    |-- element: long (containsNull = true)

+-----------+--------+--------------------+
|customer_id|order_id|             contact|
+-----------+--------+--------------------+
|       C001|    O101|[9000010000, 9000...|
+-----------+--------+--------------------+



In [ ]:

root
 |-- contact: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_line_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: double (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- qty: long (nullable = true)

In [87]:
_schema="contact array<long>,customer_id string, order_id string, order_line_items array<struct<amount double,item_id string,qty long>>"

df_with_schema=spark.read.format("json").schema(_schema).load("input/order_singleline.json")
df_with_schema.printSchema()
df_with_schema.show(truncate=False)

root
 |-- contact: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_line_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: double (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- qty: long (nullable = true)

+------------------------+-----------+--------+------------------------------------+
|contact                 |customer_id|order_id|order_line_items                    |
+------------------------+-----------+--------+------------------------------------+
|[9000010000, 9000010001]|C001       |O101    |[{102.45, I001, 6}, {2.01, I003, 2}]|
+------------------------+-----------+--------+------------------------------------+



In [ ]:
from pyspark.sql.functions import lit,array,explode,col
df_dummy=spark.range(10)
df_dummy=df_dummy.withColumn("myarray",array(lit(10),lit(20),lit(30)))
df_dummy.withColumn("explodeddummy",explode(col("myarray"))).drop("myarray").show()


In [53]:
df.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"order_id":"O101","customer_id":"C001","order_line_items":[{"item_id":"I001","qty":6,"amount":102.45},{"item_id":"I003","qty":2,"amount":2.01}],"contact":[9000010000,9000010001]}|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [88]:
from pyspark.sql.functions import from_json
_schema="contact array<long>,customer_id string, order_id string, order_line_items array<struct<amount double,item_id string,qty long>>"
df_expanded=df.withColumn("parsed",from_json(df.value,_schema))

df_expanded.printSchema()
df_expanded.show(truncate=False)

root
 |-- value: string (nullable = true)
 |-- parsed: struct (nullable = true)
 |    |-- contact: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- customer_id: string (nullable = true)
 |    |-- order_id: string (nullable = true)
 |    |-- order_line_items: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- amount: double (nullable = true)
 |    |    |    |-- item_id: string (nullable = true)
 |    |    |    |-- qty: long (nullable = true)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------+
|value                                                                                                                                                                              |parsed                          

In [58]:
%%markdown

from_json is used to convert the string data type into JSON datatype

to_json is used to convert the json datatype to string




from_json is used to convert the string data type into JSON datatype

to_json is used to convert the json datatype to string



In [89]:
from pyspark.sql.functions import to_json

df_unparsed=df_expanded.withColumn("againString",to_json(df_expanded.parsed))
df_unparsed.printSchema()

root
 |-- value: string (nullable = true)
 |-- parsed: struct (nullable = true)
 |    |-- contact: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- customer_id: string (nullable = true)
 |    |-- order_id: string (nullable = true)
 |    |-- order_line_items: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- amount: double (nullable = true)
 |    |    |    |-- item_id: string (nullable = true)
 |    |    |    |-- qty: long (nullable = true)
 |-- againString: string (nullable = true)



In [90]:
df_unparsed.select("againString").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|againString                                                                                                                                                                        |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"contact":[9000010000,9000010001],"customer_id":"C001","order_id":"O101","order_line_items":[{"amount":102.45,"item_id":"I001","qty":6},{"amount":2.01,"item_id":"I003","qty":2}]}|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [64]:
#getting the value only from parsed column
df_1=df_expanded.select("parsed")
df_1.show()

+--------------------+
|              parsed|
+--------------------+
|{[9000010000, 900...|
+--------------------+



In [91]:
df_1=df_expanded.select("parsed.*")
df_1.printSchema()
df_1.show(truncate=False)

root
 |-- contact: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_line_items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- amount: double (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- qty: long (nullable = true)

+------------------------+-----------+--------+------------------------------------+
|contact                 |customer_id|order_id|order_line_items                    |
+------------------------+-----------+--------+------------------------------------+
|[9000010000, 9000010001]|C001       |O101    |[{102.45, I001, 6}, {2.01, I003, 2}]|
+------------------------+-----------+--------+------------------------------------+



In [92]:
from pyspark.sql.functions import explode
df_2=df_1.withColumn("expanded_line_items",explode("order_line_items"))
df_2.show(truncate=False)

+------------------------+-----------+--------+------------------------------------+-------------------+
|contact                 |customer_id|order_id|order_line_items                    |expanded_line_items|
+------------------------+-----------+--------+------------------------------------+-------------------+
|[9000010000, 9000010001]|C001       |O101    |[{102.45, I001, 6}, {2.01, I003, 2}]|{102.45, I001, 6}  |
|[9000010000, 9000010001]|C001       |O101    |[{102.45, I001, 6}, {2.01, I003, 2}]|{2.01, I003, 2}    |
+------------------------+-----------+--------+------------------------------------+-------------------+



In [101]:
df_3=df_2.select("contact","customer_id","order_id","expanded_line_items.*")       
df_3.show()
df_3.printSchema()

+--------------------+-----------+--------+------+-------+---+
|             contact|customer_id|order_id|amount|item_id|qty|
+--------------------+-----------+--------+------+-------+---+
|[9000010000, 9000...|       C001|    O101|102.45|   I001|  6|
|[9000010000, 9000...|       C001|    O101|  2.01|   I003|  2|
+--------------------+-----------+--------+------+-------+---+

root
 |-- contact: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- item_id: string (nullable = true)
 |-- qty: long (nullable = true)



In [96]:
df_4=df_3.withColumn("contact_expanded",explode("contact")).drop(col("contact"))
df_4.show()

+-----------+--------+------+-------+---+----------------+
|customer_id|order_id|amount|item_id|qty|contact_expanded|
+-----------+--------+------+-------+---+----------------+
|       C001|    O101|102.45|   I001|  6|      9000010000|
|       C001|    O101|102.45|   I001|  6|      9000010001|
|       C001|    O101|  2.01|   I003|  2|      9000010000|
|       C001|    O101|  2.01|   I003|  2|      9000010001|
+-----------+--------+------+-------+---+----------------+



In [97]:
df_4.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- item_id: string (nullable = true)
 |-- qty: long (nullable = true)
 |-- contact_expanded: long (nullable = true)



In [98]:
df_4.show()

+-----------+--------+------+-------+---+----------------+
|customer_id|order_id|amount|item_id|qty|contact_expanded|
+-----------+--------+------+-------+---+----------------+
|       C001|    O101|102.45|   I001|  6|      9000010000|
|       C001|    O101|102.45|   I001|  6|      9000010001|
|       C001|    O101|  2.01|   I003|  2|      9000010000|
|       C001|    O101|  2.01|   I003|  2|      9000010001|
+-----------+--------+------+-------+---+----------------+



In [108]:
from pyspark.sql.functions import explode
df_2=df_1.withColumn("expanded_line_items",explode("order_line_items")).withColumn("exploded_contacts",explode(col("contact"))).drop("contact","order_line_items")
df_2.select("*","expanded_line_items.*").show()
df_2.show(truncate=False)

+-----------+--------+-------------------+-----------------+------+-------+---+
|customer_id|order_id|expanded_line_items|exploded_contacts|amount|item_id|qty|
+-----------+--------+-------------------+-----------------+------+-------+---+
|       C001|    O101|  {102.45, I001, 6}|       9000010000|102.45|   I001|  6|
|       C001|    O101|  {102.45, I001, 6}|       9000010001|102.45|   I001|  6|
|       C001|    O101|    {2.01, I003, 2}|       9000010000|  2.01|   I003|  2|
|       C001|    O101|    {2.01, I003, 2}|       9000010001|  2.01|   I003|  2|
+-----------+--------+-------------------+-----------------+------+-------+---+

+-----------+--------+-------------------+-----------------+
|customer_id|order_id|expanded_line_items|exploded_contacts|
+-----------+--------+-------------------+-----------------+
|C001       |O101    |{102.45, I001, 6}  |9000010000       |
|C001       |O101    |{102.45, I001, 6}  |9000010001       |
|C001       |O101    |{2.01, I003, 2}    |9000010000  